# Notebook
In this notebook, we present the major experimental findings.

## PE-UCRL in a polarising recommender engine

### Initialisations

Choose the configurations file by altering the path below.

In [6]:
config_file_path = "config_files/peucrl_polarisation_1.json"

To run the notebook cell below, ```PilotExperimentation``` and ```gym_cellular``` have to be cloned as two separate directories in the same directory.
The notebook cell reads necessary files, imports necessary packages, and instantiates the environment and agent classes.

In [7]:
# import packages
from agents import PeUcrlAgent
from json import load
import gymnasium as gym
!cd ..; pip3 install -e gym-cellular -q
import gym_cellular
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pprint

# import configurations
config_file = open(config_file_path, 'r')
config = load(config_file)

# instantiate environment
env = gym.make(
    config["environment_version"],
    n_users=config["n_users"],
    n_user_states=config["n_user_states"],
    n_recommendations=config["n_recommendations"],
    n_moderators=config["n_moderators"],
    seed=config["environment_seed"],
)

#def reward_function(x,y):
#    return 0

# instantiate agent
agt = PeUcrlAgent(
    confidence_level=config["confidence_level"],
    accuracy=config["accuracy"],
    n_cells=config["n_users"],
    n_intracellular_states=config["n_user_states"] * 2,
    cellular_encoding=env.cellular_encoding,
    n_intracellular_actions=config["n_recommendations"],
    cellular_decoding=env.cellular_decoding,
    reward_function=env.tabular_reward_function,
    cell_classes=config["cell_classes"],
    cell_labelling_function=config["cell_labelling_function"],
    regulatory_constraints=config["regulatory_constraints"],
    initial_policy=env.get_initial_policy(),
)

pprint(config)

AssertionError: 

### Explore

The agent is run in the environment in the notebook cell below, and data is collected.

In [ ]:
# initialise environment
previous_state, info = env.reset(seed=config["reset_seed"])

# initialise data-saving structures
reward_cumulation = np.zeros(config["max_time_steps"])
side_effects_incidence = np.zeros(config["max_time_steps"])
ns_between_time_steps = np.zeros(config["max_time_steps"])
ns_between_episodes = np.zeros(config["max_time_steps"])

for time_step in range(config["max_time_steps"]):

    # interact
    action = agt.sample_action(previous_state)
    current_state, reward, terminated, truncated, info = env.step(action)
    agt.update(current_state, reward, info["side_effects"])

    # save data
    reward_cumulation[time_step] = reward
    side_effects_incidence[time_step] = env.get_side_effects_incidence()
    ns_between_time_steps[time_step] = agt.get_ns_between_time_steps()
    ns_between_episodes[time_step] = agt.get_ns_between_episodes()

    if terminated or truncated:
        break

    previous_state = current_state

### Results

In the notebook cell below, side effects incidence and reward respectively are plotted against the number of time steps.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)
x = np.arange(0, config["max_time_steps"])
ax2.set_xlabel("time step")
ax1.set_ylabel("side effects incidence")
y = side_effects_incidence
ax1.plot(x, y)
ax2.set_ylabel("reward")
y = reward_cumulation
ax2.plot(x, y)

plt.show(fig)

In the notebook cell below, a table is printed showing the time to perform the between time step computations as well as the between episodes computations.

In [ ]:
table = [
    [np.mean(ns_between_time_steps), np.std(ns_between_time_steps), config["max_time_steps"]],
    [np.nanmean(ns_between_episodes), np.nanstd(ns_between_episodes), np.count_nonzero(~np.isnan(ns_between_episodes))],
]
df = pd.DataFrame(table, columns=["mean (ns)", "standard devation (ns)", "number of samples"], index=["between time steps", "between episodes"])

print(df)